# Matching shot_number between GEDI L4A & ALS
We will match shot numbers in the Level 2A GEDI data with the ALS crossover data.

In [1]:
import os
import random

import h5py
import numpy as np
import pandas as pd
import geopandas as gp
# import geoviews as gv
# from geoviews import opts, tile_sources as gvts
# import holoviews as hv
# gv.extension('bokeh', 'matplotlib')
from shapely.geometry import Point
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

import matplotlib
import matplotlib.pyplot as plt
# import PyQt6

In [2]:
inDir = os.getcwd() + "/Input_files"
print(inDir)
input_file_names = [g for g in os.listdir(inDir) if g.startswith('GEDI04_A') and g.endswith('.h5')]  # List all GEDI level 2 files in inDir
input_file_names

/oscar/scratch/jzhu118/GEDI_Outlier_Detection_OSCAR/Input_files


['GEDI04_A_2021009022644_O11762_03_T01637_02_002_02_V002.h5',
 'GEDI04_A_2022106075705_O18927_04_T10647_02_003_01_V002.h5']

### Loading files with all information into a huge Pandas dataframe

#### Preprocessing: Get files and sort by beam

In [3]:
input_files = []
files_to_beams = dict()
for n in input_file_names:
    file_path = os.path.join(inDir, n)  # Select an example file
    file = h5py.File(file_path, 'r')
    input_files.append(file)
    
    print('Loading file: ' + n)
    print('The file contains the following groups: ' + str(list(file.keys())))
    
    print("The file's metadata contains the following attributes: ")
    for g in file['METADATA']['DatasetIdentification'].attrs: print(g)
    
    beamNames = [g for g in file.keys() if g.startswith('BEAM')]
    files_to_beams[file] = beamNames
    
    print("The file contains the following beams: ")
    for b in beamNames:
        print(f"{b} is a {file[b].attrs['description']}")

Loading file: GEDI04_A_2021009022644_O11762_03_T01637_02_002_02_V002.h5
The file contains the following groups: ['ANCILLARY', 'BEAM0000', 'BEAM0001', 'BEAM0010', 'BEAM0011', 'BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011', 'METADATA']
The file's metadata contains the following attributes: 
PGEVersion
VersionID
abstract
characterSet
creationDate
credit
fileName
gedi_l4a_githash
language
originatorOrganizationName
purpose
shortName
spatialRepresentationType
status
topicCategory
uuid
The file contains the following beams: 
BEAM0000 is a Coverage beam
BEAM0001 is a Coverage beam
BEAM0010 is a Coverage beam
BEAM0011 is a Coverage beam
BEAM0101 is a Full power beam
BEAM0110 is a Full power beam
BEAM1000 is a Full power beam
BEAM1011 is a Full power beam
Loading file: GEDI04_A_2022106075705_O18927_04_T10647_02_003_01_V002.h5
The file contains the following groups: ['ANCILLARY', 'BEAM0000', 'BEAM0001', 'BEAM0010', 'BEAM0011', 'BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011', 'METADATA']
The file'

#### Helper functions for the main loop

In [4]:
# -----------------------------------------------------------------------------
# HELPER FUNCTIONS
# -----------------------------------------------------------------------------

def collect_all_datasets(file_path):
    """
    Recursively collects all dataset paths within the HDF5 file.
    Returns a list of dataset path strings.
    """
    dataset_paths = []
    
    def visitor_func(name, node):
        if isinstance(node, h5py.Dataset):
            dataset_paths.append(name)
    
    with h5py.File(file_path, 'r') as f:
        f.visititems(visitor_func)
    
    return dataset_paths

def get_dataset_by_name(h5_file, beam_ds, name):
    """
    Given an open HDF5 file (h5_file) and a list of dataset paths (beam_ds),
    returns the data for the first dataset whose path ends with '/{name}'.
    If not found, returns None.
    """
    candidates = [ds for ds in beam_ds if ds.endswith(f'/{name}')]
    if not candidates:
        print(f"Warning: No dataset ending with '/{name}' found.")
        return None
    dataset_path = candidates[0]
    return h5_file[dataset_path][()]  # Read dataset into memory

def load_attributes_to_dict(txt_file_path):
    """
    Reads attribute names from a text file (one per line) and returns a dictionary.
    This dictionary maps the human-friendly column name to the dataset suffix.
    """
    with open(txt_file_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    
    attr_dict = {}
    for raw_attr in lines:
        attr_dict[raw_attr] = raw_attr  # Direct mapping for simplicity
    return attr_dict

In [5]:
# -----------------------------------------------------------------------------
# LOAD ALS CROSSOVER DATA
# -----------------------------------------------------------------------------

als_csv_path = os.path.join('Input_files', 'GEDI_ALSCROSSOVERS.csv')
als_df = pd.read_csv(als_csv_path, dtype={'shot_number': str})

# Ensure shot numbers are of a consistent type (adjust if necessary)
als_df['shot_number'] = als_df['shot_number'].astype(str)
als_shot_set = set(als_df['shot_number'])
print(f"Loaded ALS crossover data with {len(als_shot_set)} unique shot numbers.")

Loaded ALS crossover data with 76778 unique shot numbers.


### THIS CELL IS FOR TESTING ONLY!

In [6]:
# Read the parquet file containing all GEDI waveform shots
gedi_df = pd.read_parquet('L4A_raw.parquet')

# Assume the shot numbers are in a column named 'shot_number'
# Randomly sample 10% of the shot numbers for testing purposes.
# Set a random state for reproducibility.
sample_fraction = 0.1
random_state = 42

als_shot_series = gedi_df['Shot Number'].sample(frac=sample_fraction, random_state=random_state)

# Convert shot numbers to strings for consistent matching later on
als_shot_set = set(als_shot_series.astype(str))

print(f"Selected {len(als_shot_set)} shot numbers for als_shot_set.")

Selected 233736 shot numbers for als_shot_set.


In [7]:
# -----------------------------------------------------------------------------
# LOAD ATTRIBUTE DICTIONARY (including Beam and Channel, among others)
# -----------------------------------------------------------------------------

txt_path = 'L4A_features.txt'
ATTRIBUTES_TO_LOAD = load_attributes_to_dict(txt_path)
print("Loaded attributes:", ATTRIBUTES_TO_LOAD)

# Define the list of xvar metrics to extract (each is 2D with shape (# shots, 4))
xvar_features = ['xvar', 'xvar_a1', 'xvar_a2', 'xvar_a3', 
                 'xvar_a4', 'xvar_a5', 'xvar_a6', 'xvar_a10']

Loaded attributes: {'agbd': 'agbd', 'agbd_a1': 'agbd_a1', 'agbd_a10': 'agbd_a10', 'agbd_a2': 'agbd_a2', 'agbd_a3': 'agbd_a3', 'agbd_a4': 'agbd_a4', 'agbd_a5': 'agbd_a5', 'agbd_a6': 'agbd_a6', 'agbd_pi_lower': 'agbd_pi_lower', 'agbd_pi_lower_a1': 'agbd_pi_lower_a1', 'agbd_pi_lower_a10': 'agbd_pi_lower_a10', 'agbd_pi_lower_a2': 'agbd_pi_lower_a2', 'agbd_pi_lower_a3': 'agbd_pi_lower_a3', 'agbd_pi_lower_a4': 'agbd_pi_lower_a4', 'agbd_pi_lower_a5': 'agbd_pi_lower_a5', 'agbd_pi_lower_a6': 'agbd_pi_lower_a6', 'agbd_pi_upper': 'agbd_pi_upper', 'agbd_pi_upper_a1': 'agbd_pi_upper_a1', 'agbd_pi_upper_a10': 'agbd_pi_upper_a10', 'agbd_pi_upper_a2': 'agbd_pi_upper_a2', 'agbd_pi_upper_a3': 'agbd_pi_upper_a3', 'agbd_pi_upper_a4': 'agbd_pi_upper_a4', 'agbd_pi_upper_a5': 'agbd_pi_upper_a5', 'agbd_pi_upper_a6': 'agbd_pi_upper_a6', 'agbd_se': 'agbd_se', 'agbd_se_a1': 'agbd_se_a1', 'agbd_se_a10': 'agbd_se_a10', 'agbd_se_a2': 'agbd_se_a2', 'agbd_se_a3': 'agbd_se_a3', 'agbd_se_a4': 'agbd_se_a4', 'agbd_se_a5'

#### Main loop, collects all target features and puts them inside a Pandas dataframe.
WARNING: Despite my best attempts at optimization, this is *extremely* memory intensive even for one 5GB HDF5 file. Only run on a supercomputer.

In [9]:
# -------------------------
# MAIN EXTRACTION LOOP FOR L4A FILES WITH ALS MATCHING
# -------------------------

# List to hold DataFrames for each beam
dataframes = []

for f in input_files:
    print(f"\nProcessing L4A file: {f.filename}")
    
    # Collect all dataset paths in the current file
    all_ds_for_file = collect_all_datasets(f.filename)
    
    # Process each beam in the file
    for b in files_to_beams[f]:
        print(f"  Processing beam: {b}")
        
        # Filter dataset paths to only those belonging to the current beam
        beam_ds = [ds for ds in all_ds_for_file if b in ds]
        
        # 1) Retrieve the shot_number dataset (1D array; one entry per shot)
        shotNums = get_dataset_by_name(f, beam_ds, 'shot_number')
        if shotNums is None:
            print(f"    Warning: No 'shot_number' dataset found for beam {b}. Skipping beam.")
            continue
        
        # Convert shot numbers to a NumPy array of strings (for consistent matching)
        shotNums = np.array(shotNums).astype(str)
        total_shots = len(shotNums)
        print(f"    Total number of shots: {total_shots}")
        
        # --- MATCHING WITH ALS DATA ---
        # Determine matching indices using the als_shot_set
        matching_indices = np.where(np.isin(shotNums, list(als_shot_set)))[0]
        if matching_indices.size == 0:
            print(f"    No matching shot numbers found for beam {b}. Skipping beam.")
            continue
        filtered_shot_count = matching_indices.size
        print(f"    Found {filtered_shot_count} matching shots.")
        
        # Filter shot numbers and build constant columns for the matching shots only
        shotNums_filtered = shotNums[matching_indices]
        file_name_col = [os.path.basename(f.filename)] * filtered_shot_count
        beam_name_col = [b] * filtered_shot_count
        
        # 2) Extract all 1D features specified in the L4A feature list, filtering by matching indices
        attr_data = {}
        for col_label, ds_suffix in ATTRIBUTES_TO_LOAD.items():
            data_array = get_dataset_by_name(f, beam_ds, ds_suffix)
            if data_array is None:
                print(f"    Warning: Missing dataset for {ds_suffix}. Filling with None.")
                attr_data[col_label] = np.full(total_shots, None)[matching_indices]
            else:
                data_array = np.array(data_array)
                if len(data_array) != total_shots:
                    print(f"    Warning: Row count mismatch for {col_label} in beam {b}.")
                # Filter to keep only matching shots
                attr_data[col_label] = data_array[matching_indices]
        
        # 3) Extract and process each xvar feature (each is 2D with shape (# shots, 4)), filtering by matching indices
        xvar_data = {}
        for feature in xvar_features:
            data_array = get_dataset_by_name(f, beam_ds, feature)
            if data_array is None:
                print(f"    Warning: Missing xvar feature: {feature}. Filling with NaNs.")
                for i in range(1, 5):
                    xvar_data[f"{feature}_{i}"] = np.full(total_shots, np.nan)[matching_indices]
            else:
                data_array = np.array(data_array)
                # Check if transposition is needed: expected shape is (total_shots, 4)
                if data_array.shape[0] == 4 and data_array.shape[1] == total_shots:
                    data_array = data_array.T
                elif data_array.shape[0] != total_shots or data_array.shape[1] != 4:
                    print(f"    Warning: Unexpected shape for {feature}. Expected (# shots, 4). Got {data_array.shape}.")
                # Now, for each of the 4 columns, filter by matching_indices
                for i in range(4):
                    col_name = f"{feature}_{i+1}"
                    xvar_data[col_name] = data_array[:, i][matching_indices]
        
        # 4) Construct the DataFrame for the current beam by combining all columns for matching shots
        df = pd.DataFrame({
            'File Name': file_name_col,
            'Beam Name': beam_name_col,
            'Shot Number': shotNums_filtered,
            **attr_data,
            **xvar_data
        })
        
        dataframes.append(df)
        print(f"    Processed beam {b} with {filtered_shot_count} matching shots.")

# Optionally, concatenate all beam DataFrames into one complete DataFrame:
if dataframes:
    complete_df = pd.concat(dataframes, ignore_index=True)
    
    # 5) One-hot encode categorical features, e.g., Beam Name and channel (if present)
    if 'beam' in complete_df.columns:
        complete_df = pd.get_dummies(complete_df, columns=['beam'], prefix='beam')
    if 'channel' in complete_df.columns:
        complete_df = pd.get_dummies(complete_df, columns=['channel'], prefix='channel')
    if 'region_class' in complete_df.columns:
        complete_df = pd.get_dummies(complete_df, columns=['region_class'], prefix='region')
    if 'pft_class' in complete_df.columns:
        complete_df = pd.get_dummies(complete_df, columns=['pft_class'], prefix='pft')
    print('Encoding complete')

    print(f"\nFinal DataFrame shape: {complete_df.shape}")
else:
    complete_df = pd.DataFrame()
    print("\nNo data extracted from L4A files.")


Processing L4A file: /oscar/scratch/jzhu118/GEDI_Outlier_Detection_OSCAR/Input_files/GEDI04_A_2021009022644_O11762_03_T01637_02_002_02_V002.h5
  Processing beam: BEAM0000
    Total number of shots: 167298
    Found 16615 matching shots.
    Processed beam BEAM0000 with 16615 matching shots.
  Processing beam: BEAM0001
    Total number of shots: 167288
    Found 16882 matching shots.
    Processed beam BEAM0001 with 16882 matching shots.
  Processing beam: BEAM0010
    Total number of shots: 167310
    Found 16855 matching shots.
    Processed beam BEAM0010 with 16855 matching shots.
  Processing beam: BEAM0011
    Total number of shots: 167328
    Found 16633 matching shots.
    Processed beam BEAM0011 with 16633 matching shots.
  Processing beam: BEAM0101
    Total number of shots: 167265
    Found 16831 matching shots.
    Processed beam BEAM0101 with 16831 matching shots.
  Processing beam: BEAM0110
    Total number of shots: 167030
    Found 16633 matching shots.
    Processed bea

In [12]:
# Optionally, save the complete DataFrame for further processing:
# Get a list of column names
columns = complete_df.columns

# Find duplicates
duplicate_columns = columns[columns.duplicated()]
print("Duplicate columns:", duplicate_columns)
print('There are ' + str(columns.duplicated().sum()) + ' duplicated columns.')
print('NaN value count:\n')
print(complete_df.isnull().sum())
complete_df.describe()

# complete_df.to_parquet("L4A_and_ALS.parquet", engine="pyarrow", compression="snappy")
# print('Successfully wrote to a .parquet file!')

Duplicate columns: Index([], dtype='object')
There are 0 duplicated columns.
NaN value count:

File Name             0
Shot Number           0
agbd                  0
agbd_a1               0
agbd_a10              0
                  ...  
channel_3        204682
Beam_BEAM1000    204507
channel_4        204507
Beam_BEAM1011    204672
channel_5        204672
Length: 218, dtype: int64


,agbd,agbd_a1,agbd_a10,agbd_a2,agbd_a3,agbd_a4,agbd_a5,agbd_a6,agbd_pi_lower,agbd_pi_lower_a1,...,xvar_a5_3,xvar_a5_4,xvar_a6_1,xvar_a6_2,xvar_a6_3,xvar_a6_4,xvar_a10_1,xvar_a10_2,xvar_a10_3,xvar_a10_4
count,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000,...,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000,233736.000000
mean,-5723.515137,-5729.544922,-5699.986816,-5704.903320,-5724.968750,-5733.004395,-5694.802734,-5708.046875,-8903.421875,-8916.979492,...,-5717.515625,-5719.214355,-5717.739746,-5719.590820,-5720.391113,-5722.080566,-5714.857910,-5716.710449,-5717.522461,-5719.214355
std,4960.680664,4959.826172,4970.760254,4967.098145,4958.889648,4955.729980,4975.967773,4959.737793,3130.763428,3111.988525,...,4944.986328,4942.834961,4949.347656,4952.636719,4951.495605,4946.908203,4949.427246,4946.170898,4944.979004,4942.834961
min,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
25%,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
50%,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
75%,1.459848,1.453466,0.970350,0.970350,0.977234,1.214101,1.061925,0.980942,-9999.000000,-9999.000000,...,0.000000,0.000000,10.033444,0.000000,0.000000,0.000000,10.031450,0.000000,0.000000,0.000000
max,4549.524902,4549.524902,4915.053711,4917.447754,4538.434570,2850.412842,5066.130371,4919.843262,3862.254395,3862.254395,...,13.695985,0.000000,15.383758,14.402083,12.912785,0.000000,15.381159,14.402083,13.406714,0.000000


#### Feature engineering
1. Adding Z-Scores for five most relevant RH metrics
2. Creating RH50 / RH100 ratio
3. Adding RH95 - RH50
4. Adding "Missingness" - the number of NaNs in each row

In [ ]:
num_zeros = (complete_df['RH_100'] == 0).sum()
print(f"Number of zeros in RH_100: {num_zeros}")

In [ ]:
'''Add Z-Scores of the five RH metrics'''
rh_nums = [25, 50, 75, 85, 95, 100]
for i in rh_nums:
    col_name = f'RH_{i}'
    complete_df[f'{col_name} Z Score'] = (complete_df[col_name] - complete_df[col_name].mean()) / complete_df[col_name].std()

complete_df

In [ ]:
'''Adding the RH_50 / RH_100 feature'''
complete_df['RH_50_v_100'] = complete_df['RH_50'] / complete_df['RH_100']
complete_df

In [ ]:
'''Adding the (RH95 - RH50) feature'''
rh_50 = complete_df['RH_50']
rh_95 = complete_df['RH_95']
complete_df['RH_95_minus_50'] = (rh_95 - rh_50)
complete_df

In [ ]:
'''Adding the 'Missingness' feature'''
# Count the number of NaNs in each row
complete_df['Missingness'] = complete_df.isna().sum(axis=1)


# Optionally, inspect how many rows have missing data
print('Number of NaNs:')
print(complete_df['Missingness'].value_counts())
complete_df

In [ ]:
'''Optional: Save to a parquet file'''
# Get a list of column names
columns = complete_df.columns

# Find duplicates
duplicate_columns = columns[columns.duplicated()]
print("Duplicate columns:", duplicate_columns)
print('There are ' + str(columns.duplicated().sum()) + ' duplicated columns.')
print('NaN value count:\n')
print(complete_df.isnull().sum())
# complete_df.describe()

# Uncomment the two lines below to write to parquet
# complete_df.to_parquet("input_raw.parquet", engine="pyarrow", compression="snappy")
# print('Successfully wrote to a .parquet file!')

#### This is the filtering step for PCA if there are any NaN rows in the dataframe
Note: there should not be mnany NaN values if you accounted for the different shapes of the data inside the HDF5 files.

In [ ]:
'''Optional: load complete_df from .parquet file'''
complete_df = pd.read_parquet('input_raw.parquet', engine='pyarrow')
complete_df

In [ ]:
# from sklearn.impute import SimpleImputer

print(f'Original dataframe shape: {complete_df.shape}')

'''Filtering'''
complete_df.drop('Shot Number', axis=1, inplace=True)
print("Shot numbers dropped")
discounted_df = complete_df.dropna(axis=1)


# imputer = SimpleImputer(strategy='most_frequent')  # You can change to 'median', 'most_frequent', etc.
# discounted_df = pd.DataFrame(imputer.fit_transform(complete_df), columns=complete_df.columns)

print("Dataframe shape after dropping NaN columns:", discounted_df.shape)

# Step 1: Separate features and target (if applicable)
# Exclude non-numeric columns if present
filtered_df = discounted_df.select_dtypes(include=[np.number])
# columns_to_keep = [col for col in numeric_df.columns if 'RH' not in col]
# filtered_df = filtered_df[columns_to_keep]

print(f'Non-numeric columns removed from dataframe\nCleaned dataframe size: {filtered_df.shape}')
filtered_df

In [ ]:
# Step 2, OPTION 1: Standardize the data using StandardScalar (for demo only)
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(filtered_df)
# print(f'Data scaled\nScaled data size (ndarray): {scaled_data.shape}')

scaled_df = pd.DataFrame(scaled_data, columns=filtered_df.columns, index=filtered_df.index)
print("Made new scaled dataframe")
scaled_df
scaled_df.describe()

# Uncomment the two lines below to write scaled_df to parquet
scaled_df.to_parquet("input_standard_scaled.parquet", engine="pyarrow", compression="snappy")
print('Successfully wrote to a .parquet file!')

In [ ]:
# OPTION 2: Standardize the data using RobustScalar (this is better for outlier detection)
from sklearn.preprocessing import RobustScaler

# Assume `filtered_df` is your cleaned and filtered dataframe (all numeric columns)
robust_scaler = RobustScaler()
print("Instantiated RobustScalar")

# Fit the scaler on the dataframe and transform it
scaled_array = robust_scaler.fit_transform(filtered_df)
print("Fitted RobustScalar")

# (Optional) Create a new DataFrame with scaled values
scaled_df = pd.DataFrame(scaled_array, columns=filtered_df.columns, index=filtered_df.index)
print("Made new scaled dataframe")
scaled_df
scaled_df.describe()

# Uncomment the two lines below to write scaled_df to parquet
scaled_df.to_parquet("input_scaled.parquet", engine="pyarrow", compression="snappy")
print('Successfully wrote to a .parquet file!')